In [1]:
from __future__ import division
from api import git_access,api_access
from git_understand import git_understand_v1 as git_understand
from git_log import git2repo
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import networkx as nx
import re
from git_log import git2data,git_commit_info,release_mine,project_url_mine
import threading
from threading import Barrier
from multiprocessing import Queue
from os.path import dirname as up
import os
import platform
from commit_guru.cas_manager_v1 import *

In [2]:
if platform.system() == 'Darwin' or platform.system() == 'Linux':
    data_path = os.getcwd() + '/Test_projects.csv'
else:
    data_path = os.getcwd() + '\\Test_projects.csv'
code_path = os.getcwd()
project_list = pd.read_csv(data_path)
for i in range(project_list.shape[0]):
#     try:
    print("I am here")
    understand_source = []
    last_analyzed = None
    access_token = project_list.loc[i,'access_token']
    repo_owner = project_list.loc[i,'repo_owner']
    source_type = project_list.loc[i,'source_type']
    git_url = project_list.loc[i,'git_url']
    api_base_url = project_list.loc[i,'api_base_url']
    repo_name = project_list.loc[i,'repo_name']
    repo_lang = project_list.loc[i,'lang']
    #git_data = git2data.git2data(access_token,repo_owner,source_type,git_url,api_base_url,repo_name)
    git_data = project_url_mine.git2data(access_token,repo_owner,source_type,git_url,api_base_url,repo_name)
    git_data.create_data()
    break
#     matrix = get_matrix.get_defective_pair_metrics()
#     except as e:
#         print("Exception occured for ",project_list.loc[i,'git_url'])
#         print(e)
#         continue

I am here
url creation http://api.github.com/search/repositories?q=language:Java+forks:0..50
This is paged: http://api.github.com/search/repositories?q=language:Java+forks:0..50&page=1
30
This is paged: http://api.github.com/search/repositories?q=language:Java+forks:0..50&page=2
30
This is paged: http://api.github.com/search/repositories?q=language:Java+forks:0..50&page=3
30
This is paged: http://api.github.com/search/repositories?q=language:Java+forks:0..50&page=4
30
This is paged: http://api.github.com/search/repositories?q=language:Java+forks:0..50&page=5
30
This is paged: http://api.github.com/search/repositories?q=language:Java+forks:0..50&page=6
30
This is paged: http://api.github.com/search/repositories?q=language:Java+forks:0..50&page=7
30
This is paged: http://api.github.com/search/repositories?q=language:Java+forks:0..50&page=8
30
This is paged: http://api.github.com/search/repositories?q=language:Java+forks:0..50&page=9
30
This is paged: http://api.github.com/search/reposito

2020-01-21 12:46:39 WARNING: API rate limit exceeded for uri: http://api.github.com/search/repositories?q=language:Java+forks:50..100&page=24
2020-01-21 12:46:39 WARNING: API rate limit exceeded for uri: http://api.github.com/search/repositories?q=language:Java+forks:50..100&page=24. Waiting for 0 mins 12 seconds. Restarting at 21 Jan 2020 12:46:50 ...


30
This is paged: http://api.github.com/search/repositories?q=language:Java+forks:50..100&page=24
1579628810 1579628799.0 12.0


2020-01-21 12:46:51 WARNING: API rate limit exceeded for uri: http://api.github.com/search/repositories?q=language:Java+forks:50..100&page=24
2020-01-21 12:46:51 WARNING: API rate limit exceeded for uri: http://api.github.com/search/repositories?q=language:Java+forks:50..100&page=24. Waiting for 1 mins 1 seconds. Restarting at 21 Jan 2020 12:47:51 ...


1579628871 1579628811.0 61.0
30
This is paged: http://api.github.com/search/repositories?q=language:Java+forks:50..100&page=25
30
This is paged: http://api.github.com/search/repositories?q=language:Java+forks:50..100&page=26
30
This is paged: http://api.github.com/search/repositories?q=language:Java+forks:50..100&page=27
30
This is paged: http://api.github.com/search/repositories?q=language:Java+forks:50..100&page=28
30
This is paged: http://api.github.com/search/repositories?q=language:Java+forks:50..100&page=29
30
This is paged: http://api.github.com/search/repositories?q=language:Java+forks:50..100&page=30
30
This is paged: http://api.github.com/search/repositories?q=language:Java+forks:50..100&page=31
30
This is paged: http://api.github.com/search/repositories?q=language:Java+forks:50..100&page=32
30
This is paged: http://api.github.com/search/repositories?q=language:Java+forks:50..100&page=33
30
This is paged: http://api.github.com/search/repositories?q=language:Java+forks:50..100

...

1579711799 1579711739.0 61.0
6
updating fork
url creation http://api.github.com/search/repositories?q=language:JavaScript+forks:3300..3350
This is paged: http://api.github.com/search/repositories?q=language:JavaScript+forks:3300..3350&page=1
2
updating fork
url creation http://api.github.com/search/repositories?q=language:JavaScript+forks:3350..3400
This is paged: http://api.github.com/search/repositories?q=language:JavaScript+forks:3350..3400&page=1
7
updating fork
url creation http://api.github.com/search/repositories?q=language:JavaScript+forks:3400..3450
This is paged: http://api.github.com/search/repositories?q=language:JavaScript+forks:3400..3450&page=1
4
updating fork
url creation http://api.github.com/search/repositories?q=language:JavaScript+forks:3450..3500
This is paged: http://api.github.com/search/repositories?q=language:JavaScript+forks:3450..3500&page=1
5
updating fork
url creation http://api.github.com/search/repositories?q=language:JavaScript+forks:3500..3550
This is p

2020-01-22 11:50:19 WARNING: API rate limit exceeded for uri: http://api.github.com/search/repositories?q=language:JavaScript+forks:4750..4800&page=1
2020-01-22 11:50:19 WARNING: API rate limit exceeded for uri: http://api.github.com/search/repositories?q=language:JavaScript+forks:4750..4800&page=1. Waiting for 0 mins 44 seconds. Restarting at 22 Jan 2020 11:51:02 ...


1
updating fork
url creation http://api.github.com/search/repositories?q=language:JavaScript+forks:4750..4800
This is paged: http://api.github.com/search/repositories?q=language:JavaScript+forks:4750..4800&page=1
1579711862 1579711819.0 44.0


2020-01-22 11:51:05 WARNING: API rate limit exceeded for uri: http://api.github.com/search/repositories?q=language:JavaScript+forks:4750..4800&page=1
2020-01-22 11:51:05 WARNING: API rate limit exceeded for uri: http://api.github.com/search/repositories?q=language:JavaScript+forks:4750..4800&page=1. Waiting for 1 mins 1 seconds. Restarting at 22 Jan 2020 11:52:05 ...


1579711925 1579711865.0 61.0
2
updating fork
url creation http://api.github.com/search/repositories?q=language:JavaScript+forks:4800..4850
This is paged: http://api.github.com/search/repositories?q=language:JavaScript+forks:4800..4850&page=1
1
updating fork
url creation http://api.github.com/search/repositories?q=language:JavaScript+forks:4850..4900
This is paged: http://api.github.com/search/repositories?q=language:JavaScript+forks:4850..4900&page=1
1
updating fork
url creation http://api.github.com/search/repositories?q=language:JavaScript+forks:4900..4950
This is paged: http://api.github.com/search/repositories?q=language:JavaScript+forks:4900..4950&page=1
3
updating fork
url creation http://api.github.com/search/repositories?q=language:JavaScript+forks:4950..5000
This is paged: http://api.github.com/search/repositories?q=language:JavaScript+forks:4950..5000&page=1
1
updating fork
url creation http://api.github.com/search/repositories?q=language:JavaScript+forks:5000..5050
This is p

In [ ]:
if platform.system() == 'Darwin' or platform.system() == 'Linux':
    data_path = os.getcwd() + '/Test_projects.csv'
else:
    data_path = os.getcwd() + '\\Test_projects.csv'
code_path = os.getcwd()
project_list = pd.read_csv(data_path)
for i in range(project_list.shape[0]):
#     try:
    print("I am here")
    understand_source = []
    last_analyzed = None
    access_token = project_list.loc[i,'access_token']
    repo_owner = project_list.loc[i,'repo_owner']
    source_type = project_list.loc[i,'source_type']
    git_url = project_list.loc[i,'git_url']
    api_base_url = project_list.loc[i,'api_base_url']
    repo_name = project_list.loc[i,'repo_name']
    repo_lang = project_list.loc[i,'lang']
    understand_source.append([1,repo_name,git_url,last_analyzed])
    understand_source_df = pd.DataFrame(understand_source,columns = ['id','name','url','last_analyzed'])
    cas_manager = CAS_Manager(understand_source_df)
    cas_manager.run()
    os.chdir(code_path)
#         git_data = git2data.git2data(access_token,repo_owner,source_type,git_url,api_base_url,repo_name)
#         #git_data = release_mine.git2data(access_token,repo_owner,source_type,git_url,api_base_url,repo_name)
#         git_data.create_data()
#         df_commit = pd.read_pickle(up(code_path) + '/data/commit/' + repo_name + '_commit.pkl')
#         df_commit_guru = pd.read_csv(up(code_path) + '/data/commit_guru/' + repo_name + '.csv')
#         df_commit_guru_subset = df_commit_guru[['commit_hash','contains_bug']].fillna(False)
#         df = pd.merge(df_commit,df_commit_guru_subset, on='commit_hash',how='inner')
#         df.to_pickle(up(code_path) + '/data/commit/' + repo_name + '_commit.pkl')
    get_matrix = git_understand.MetricsGetter(git_url,repo_name,repo_lang)
    matrix = get_matrix.get_defective_pair_metrics()
#     except as e:
#         print("Exception occured for ",project_list.loc[i,'git_url'])
#         print(e)
#         continue

In [ ]:
cas_manager = CAS_Manager()
cas_manager.start()

In [ ]:
for i in range(project_list.shape[0]):
    repo_name = project_list.loc[i,'repo_name']
    df_commit = pd.read_pickle('/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit/' + repo_name + '_commit.pkl')
    df_commit_guru = pd.read_csv('/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/' + repo_name + '.csv')
    df_commit_guru_subset = df_commit_guru[['commit_hash','contains_bug']].fillna(False)
    df = pd.merge(df_commit,df_commit_guru_subset, on='commit_hash',how='inner')
    df.to_pickle('/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit/' + repo_name + '_commit.pkl')


In [ ]:
if platform.system() == 'Darwin' or platform.system() == 'Linux':
    data_path = os.getcwd() + '/Test_projects.csv'
else:
    data_path = os.getcwd() + '\\Test_projects.csv'

project_list = pd.read_csv(data_path)
print(project_list)

for i in range(project_list.shape[0]):
    try:
        repo_url = project_list.loc[i,'git_url']
        repo_name = project_list.loc[i,'repo_name']
        repo_lang = project_list.loc[i,'lang']
        get_matrix = git_understand.MetricsGetter(repo_url,repo_name,repo_lang)
        #commits = get_matrix.read_commits()
        #matrix = get_matrix.run()
        matrix = get_matrix.get_defective_pair_metrics()
    except ValueError:
        print("Error")

In [ ]:
df_new = pd.merge(matrix,df_commit_guru, on='commit_hash',how='inner')
df_new = df_new.dropna()
df_new.to_csv('/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru_understand/uhabits.csv',index=False)

In [ ]:
matrix_1 = matrix.drop(['Name','commit_hash'],axis = 1)
matrix_1 = matrix_1.drop_duplicates()
matrix_1.shape

In [ ]:
matrix_1.columns

In [ ]:
os.getcwd()

In [ ]:
if platform.system() == 'Darwin' or platform.system() == 'Linux':
    data_path = os.getcwd() + '/Test_projects.csv'
else:
    data_path = os.getcwd() + '\\Test_projects.csv'
code_path = os.getcwd()
project_list = pd.read_csv(data_path)
for i in range(project_list.shape[0]):
#     try:
    print("I am here")
    understand_source = []
    last_analyzed = None
    access_token = project_list.loc[i,'access_token']
    repo_owner = project_list.loc[i,'repo_owner']
    source_type = project_list.loc[i,'source_type']
    git_url = project_list.loc[i,'git_url']
    api_base_url = project_list.loc[i,'api_base_url']
    repo_name = project_list.loc[i,'repo_name']
    repo_lang = project_list.loc[i,'lang']
    git_client = api_access.git_api_access(access_token,repo_owner,source_type,git_url,api_base_url,repo_name)
    langs = git_client.get_lang()
    project_list.loc[i,'lang'] = langs[0]

In [ ]:
project_list

In [ ]:
project_list.to_csv('projects.csv')

In [ ]:
df1 = pd.read_csv('p_lists.csv')
lists = df1.Projects.values.tolist()

In [ ]:
df2 = pd.read_csv('Test_projects.csv')

In [ ]:
for i in range(df2.shape[0]):
    project = df2.loc[i,'repo_name']
    if project in lists:
        df2.loc[i,'done'] = 1
    else:
        df2.loc[i,'done'] = 0

In [ ]:
df2.to_csv('Test_projects.csv')

In [ ]:
'PushSharp' in lists